In [52]:
pip install skfolio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.0/734.0 kB 24.7 MB/s eta 0:00:00


In [53]:
import numpy as np
from plotly.io import show
from sklearn.model_selection import train_test_split

from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import InverseVolatility, MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns

In [54]:
prices = load_sp500_dataset()

In [55]:
prices.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-02,0.264,4.125,4.599,0.144,4.991,14.391,1.117,3.438,3.394,2.235,6.658,4.215,0.384,4.738,1.021,3.860,3.322,0.310,3.653,4.068
1990-01-03,0.266,4.000,4.636,0.161,4.910,14.364,1.121,3.452,3.508,2.203,6.658,4.282,0.386,4.692,1.024,3.853,3.322,0.304,3.653,4.027
1990-01-04,0.267,3.938,4.537,0.159,4.847,14.283,1.128,3.459,3.522,2.192,6.621,4.215,0.397,4.646,1.041,3.777,3.322,0.301,3.634,3.987
1990-01-05,0.268,3.812,4.438,0.159,4.775,14.148,1.113,3.423,3.536,2.174,6.549,4.128,0.387,4.581,1.032,3.709,3.322,0.288,3.595,3.966
1990-01-08,0.269,3.812,4.463,0.147,4.820,14.229,1.102,3.481,3.536,2.220,6.549,4.181,0.393,4.664,1.023,3.777,3.322,0.282,3.644,4.027


In [56]:
X = prices_to_returns(prices)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)


In [57]:
print(X_train.head())

                AAPL       AMD       BAC       BBY       CVX        GE  \
Date                                                                     
1990-01-03  0.007576 -0.030303  0.008045  0.118056 -0.016229 -0.001876   
1990-01-04  0.003759 -0.015500 -0.021355 -0.012422 -0.012831 -0.005639   
1990-01-05  0.003745 -0.031996 -0.021821  0.000000 -0.014855 -0.009452   
1990-01-08  0.003731  0.000000  0.005633 -0.075472  0.009424  0.005725   
1990-01-09 -0.007435  0.016527  0.000000  0.000000 -0.007469 -0.020803   

                  HD       JNJ       JPM        KO       LLY       MRK  \
Date                                                                     
1990-01-03  0.003581  0.004072  0.033589 -0.014318  0.000000  0.015896   
1990-01-04  0.006244  0.002028  0.003991 -0.004993 -0.005557 -0.015647   
1990-01-05 -0.013298 -0.010408  0.003975 -0.008212 -0.010874 -0.020641   
1990-01-08 -0.009883  0.016944  0.000000  0.021159  0.000000  0.012839   
1990-01-09 -0.026316 -0.031026 -0.031

In [58]:
model = MeanRisk(
    risk_measure=RiskMeasure.CVAR,
    objective_function=ObjectiveFunction.MINIMIZE_RISK,
    portfolio_params=dict(name="Min CVaR"),
)
model.fit(X_train)
model.weights_

array([2.16562529e-02, 1.41742527e-10, 3.64888981e-11, 1.47358386e-02,
       1.35927671e-01, 1.91035768e-10, 3.58350612e-10, 2.10319630e-01,
       4.89793803e-11, 8.14733606e-02, 1.92817471e-02, 4.13607836e-10,
       7.79844555e-10, 1.26155697e-01, 4.04585759e-10, 1.52708811e-01,
       1.20106233e-02, 6.41726679e-03, 1.01024013e-01, 1.18289085e-01])

In [77]:
model = MeanRisk(budget=-1, min_weights=-1)
model.fit(X)
print(sum(model.weights_))
model.weights_

-0.9999999999999997


array([-0.03052686,  0.00202787,  0.02628631, -0.01301355, -0.0833489 ,
        0.00198716,  0.01371319, -0.19488496,  0.02040412, -0.12423298,
       -0.03140305, -0.02378448, -0.01970516, -0.11480775, -0.01733407,
       -0.16842916, -0.01131833, -0.01512073, -0.12382938, -0.09267928])

In [84]:
model = MeanRisk(min_weights=[0,0.1,0.15,0.2,0.25])
model.fit(X_train)
print(sum(model.weights_))
model.weights_

0.9999999999999999


array([0.19901109, 0.16357856, 0.18741035, 0.2       , 0.25      ])

In [75]:
import numpy as np

# Sample data for demonstration (replace with actual data)
np.random.seed(0)
X_train = np.random.randn(100, 5)  # 100 samples, 5 assets

# Define the variables
weights = cp.Variable(X_train.shape[1])

# Define the CVaR risk measure
alpha = 0.95
VaR = cp.Variable()
z = cp.Variable(X_train.shape[0])
risk_measure = VaR + (1 / (1 - alpha)) * cp.sum(z) / X_train.shape[0]

# Define the objective function to minimize CVaR
objective = cp.Minimize(risk_measure)

# Define the constraints
constraints = [
    weights >= 0,
    cp.sum(weights) == 1,
    z >= 0,
    z >= -X_train @ weights - VaR
]

# Create the optimization problem
problem = cp.Problem(objective, constraints)

# Check if the problem is convex
is_convex = problem.is_dcp()

print(f"Is the MeanRisk model with CVaR risk measure and MINIMIZE_RISK objective function convex? {is_convex}")

Is the MeanRisk model with CVaR risk measure and MINIMIZE_RISK objective function convex? True


In [60]:
from skfolio.optimization import EqualWeighted, MeanRisk, ObjectiveFunction
benchmark = EqualWeighted(portfolio_params=dict(name="Equal Weighted"))
# Even if `X` has no impact (as it is equal weighted), we still need to call `fit` for
# API consistency.
benchmark.fit(X_train)
benchmark.weights_

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [61]:
pred_model = model.predict(X_test)
pred_bench = benchmark.predict(X_test)


In [62]:
np.asarray(pred_model)


array([ 0.00354605,  0.00342423, -0.00105085, ...,  0.01069377,
        0.00542224, -0.01217676])

In [63]:
print(pred_model.cvar)
print(pred_bench.cvar)


0.0217415516371764
0.025061083134673378


In [64]:
population = Population([pred_model, pred_bench])

In [65]:
population.plot_composition()

In [68]:
import cvxpy as cp

# Define the variables based on the DataFrame columns
variables = {col: cp.Variable(len(returns)) for col in returns.columns}

# Define a convex objective function (e.g., minimize the sum of squares)
objective = cp.Minimize(sum(cp.sum_squares(variables[col]) for col in returns.columns))

# Define example constraints (e.g., variables should be within certain bounds)
constraints = [variables[col] >= 0 for col in returns.columns]

# Create the optimization problem
problem = cp.Problem(objective, constraints)

def might_be_convex(problem, niter=100, tol=1e-8):
    def get_values(problem):
        var_values = []
        for var in problem.variables():
            if var.is_nonneg():
                value = np.random.rand(*var.shape)
            elif var.is_nonpos():
                value = -np.random.rand(*var.shape)
            elif var.is_nsd():
                value = np.random.randn(*var.shape)
                value = -value @ value.T
            elif var.is_psd():
                value = np.random.randn(*var.shape)
                value = value @ value.T
            else:
                value = np.random.randn(*var.shape)
            var_values.append(value)
        return var_values

    def set_values(problem, var_values):
        for var, var_value in zip(problem.variables(), var_values):
            var.value = var_value

    def get_objective_constraint_values(problem, maximize=False):
        if maximize:
            return np.concatenate(
                [np.array([-problem.objective.value])]
                + [np.atleast_1d(c.expr.value) for c in problem.constraints]
            )
        else:
            return np.concatenate(
                [np.array([problem.objective.value])]
                + [np.atleast_1d(c.expr.value) for c in problem.constraints]
            )

    maximize = isinstance(problem.objective, Maximize)
    might_be_convex = True
    for _ in range(niter):
        values1 = get_values(problem)
        values2 = get_values(problem)

        set_values(problem, values1)
        f1 = get_objective_constraint_values(problem, maximize)
        set_values(problem, values2)
        f2 = get_objective_constraint_values(problem, maximize)
        set_values(problem, [(v1 + v2) / 2 for v1, v2 in zip(values1, values2)])
        f12 = get_objective_constraint_values(problem, maximize)

        if not ((f1 + f2) / 2 - f12 >= -tol).all():
            might_be_convex = False
            break

    return might_be_convex

# Check if the problem might be convex
is_convex = might_be_convex(problem)

print(f"Might be convex: {is_convex}")

Might be convex: True


In [67]:
returns = prices_to_returns(prices)

In [70]:
# Define the path_length function
def path_length(x, y):
    return np.sum(np.sqrt(np.diff(x)**2 + np.diff(y)**2))

# Define the conv_meas function
def conv_meas(x, y):
    ch = np.array(list(set(np.concatenate([np.arange(len(x)), [len(x)-1]]))))
    ch = ch[ch < len(x)]
    pl = path_length(x[ch], y[ch])
    return (path_length(x, y) - pl) / pl

# Calculate convexity measure for each pair of columns in the DataFrame
conv_measures = {}
columns = df.columns
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        col1 = columns[i]
        col2 = columns[j]
        conv_measures[f'{col1}-{col2}'] = conv_meas(df[col1], df[col2])

# Print the convexity measures
for cols, measure in conv_measures.items():
    print(f"Convexity measure for {cols}: {measure}")

Convexity measure for A-B: 0.0
Convexity measure for A-C: 0.0
Convexity measure for B-C: 0.0


In [72]:
# Define the path_length function
def path_length(x, y):
    return np.sum(np.sqrt(np.diff(x)**2 + np.diff(y)**2))

# Define the conv_meas function
def conv_meas(x, y):
    ch = np.array(list(set(np.concatenate([np.arange(len(x)), [len(x)-1]]))))
    ch = ch[ch < len(x)]
    pl = path_length(x[ch], y[ch])
    return (path_length(x, y) - pl) / pl

# Calculate convexity measure for each pair of columns in the DataFrame
conv_measures = {}
columns = prices.columns
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        col1 = columns[i]
        col2 = columns[j]
        conv_measures[f'{col1}-{col2}'] = conv_meas(prices[col1], prices[col2])

# Print the convexity measures
for cols, measure in conv_measures.items():
    print(f"Convexity measure for {cols}: {measure}")

<ipython-input-72-3f99f96a1da0>:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Convexity measure for AAPL-AMD: 0.0
Convexity measure for AAPL-BAC: 0.0
Convexity measure for AAPL-BBY: 0.0
Convexity measure for AAPL-CVX: 0.0
Convexity measure for AAPL-GE: 0.0
Convexity measure for AAPL-HD: 0.0
Convexity measure for AAPL-JNJ: 0.0
Convexity measure for AAPL-JPM: 0.0
Convexity measure for AAPL-KO: 0.0
Convexity measure for AAPL-LLY: 0.0
Convexity measure for AAPL-MRK: 0.0
Convexity measure for AAPL-MSFT: 0.0
Convexity measure for AAPL-PEP: 0.0
Convexity measure for AAPL-PFE: 0.0
Convexity measure for AAPL-PG: 0.0
Convexity measure for AAPL-RRC: 0.0
Convexity measure for AAPL-UNH: 0.0
Convexity measure for AAPL-WMT: 0.0
Convexity measure for AAPL-XOM: 0.0
Convexity measure for AMD-BAC: 0.0
Convexity measure for AMD-BBY: 0.0
Convexity measure for AMD-CVX: 0.0
Convexity measure for AMD-GE: 0.0
Convexity measure for AMD-HD: 0.0
Convexity measure for AMD-JNJ: 0.0
Convexity measure for AMD-JPM: 0.0
Convexity measure for AMD-KO: 0.0
Convexity measure for AMD-LLY: 0.0
Convexi

In [74]:
from scipy.spatial import ConvexHull
# Define the path_length function
def path_length(x, y):
    return np.sum(np.sqrt(np.diff(x)**2 + np.diff(y)**2))

# Define the conv_meas function
def conv_meas(x, y):
    yrng = [np.min(y), np.max(y)]
    xhull = np.concatenate([x, [np.max(x), np.min(x)]])
    chDown = ConvexHull(np.column_stack((xhull, np.concatenate([y, [yrng[0], yrng[0]]])))).vertices
    chUp = ConvexHull(np.column_stack((xhull, np.concatenate([y, [yrng[1], yrng[1]]])))).vertices
    chDown = np.sort(chDown[chDown < len(x)])
    chUp = np.sort(chUp[chUp < len(x)])
    path_length_down = path_length(x[chDown], y[chDown])
    path_length_up = path_length(x[chUp], y[chUp])
    return path_length_down / path_length_up

# Calculate convexity measure for each pair of columns in the DataFrame
conv_measures = {}
columns = prices.columns
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        col1 = columns[i]
        col2 = columns[j]
        conv_measures[f'{col1}-{col2}'] = conv_meas(prices[col1], prices[col2])

# Print the convexity measures
for cols, measure in conv_measures.items():
    print(f"Convexity measure for {cols}: {measure}")

<ipython-input-74-0404a95ea552>:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-74-0404a95ea552>:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Convexity measure for AAPL-AMD: 0.845246977669036
Convexity measure for AAPL-BAC: 0.9334341151643992
Convexity measure for AAPL-BBY: 1.0142692011451733
Convexity measure for AAPL-CVX: 1.7201720881789104
Convexity measure for AAPL-GE: 1.2165081200167913
Convexity measure for AAPL-HD: 0.9000253838787463
Convexity measure for AAPL-JNJ: 1.3607351561037622
Convexity measure for AAPL-JPM: 0.8860075000998965
Convexity measure for AAPL-KO: 1.5420078539139714
Convexity measure for AAPL-LLY: 1.3428408203208808
Convexity measure for AAPL-MRK: 1.3608873038851643
Convexity measure for AAPL-MSFT: 0.8184337545295216
Convexity measure for AAPL-PEP: 1.2846518571813161
Convexity measure for AAPL-PFE: 1.0799118884581345
Convexity measure for AAPL-PG: 1.167447126331089
Convexity measure for AAPL-RRC: 1.4093996242118387
Convexity measure for AAPL-UNH: 1.221998285633101
Convexity measure for AAPL-WMT: 1.1385279703754123
Convexity measure for AAPL-XOM: 1.5003002995371744
Convexity measure for AMD-BAC: 1.2494